In [1]:
%load_ext autoreload  
%autoreload 2

In [2]:
import sqlite3
from typing import Any, Optional
from pydantic import BaseModel, Extra, Field
from transformers import TrainingArguments, AutoModel, AutoTokenizer, AutoModelForSequenceClassification, get_scheduler
import numpy as np
import json
import datasets
import wandb
import torch
from tqdm import tqdm
import sys
sys.path.append('../../')
from subset_active_learning.subset_selection import select, preprocess

2022-10-03 02:16:55.262670: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.1/lib64
2022-10-03 02:16:55.262706: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
ds_name = 'sst'
test_dataset = None
eval_mapping = '[]'
num_labels = 2
valid_split = 'validation'
test_split = 'validation'

db_path = "./temp.db"
seed = 0
pool_size = 1000
search_size = 100
warmup_runs = 500
annealing_runs = 1000
wandb_project = 'sst_search'
wandb_entity = 'johntzwei'

model_card = "google/electra-small-discriminator"
pretraining = True
max_steps = 3000
eval_steps = 500
learning_rate = 1e-5
batch_size = 8
# adam should default to correct_bias = True
adam_epsilon = 1e-6
adam_beta1 = 0.9
adam_beta2 = 0.999
max_grad_norm = 1.0
warmup_ratio = 0.1
weight_decay = 0.01

In [4]:
ds_name = 'mnli'
test_dataset = 'hans'
eval_mapping = '[0, 1, 1]'
num_labels = 3
valid_split = 'validation_matched'
test_split = 'train'

db_path = "./hans_temp.db"
seed = 0
pool_size = 2000
search_size = 200
warmup_runs = 100
annealing_runs = 2000
wandb_project = 'mnli_search'
wandb_entity = 'johntzwei'

max_steps = 30

In [5]:
training_args = select.SubsetTrainingArguments(model_card=model_card,
                                               num_labels=num_labels,
                                               eval_mapping=json.loads(eval_mapping),
                                               pretraining=pretraining,
                                               max_steps=max_steps,
                                               eval_steps=eval_steps,
                                               learning_rate=learning_rate,
                                               batch_size=batch_size,
                                               adam_epsilon=adam_epsilon,
                                               adam_beta1=adam_beta1,
                                               adam_beta2=adam_beta2,
                                               max_grad_norm=max_grad_norm,
                                               warmup_ratio=warmup_ratio,
                                               weight_decay=weight_decay)
searching_args = select.SubsetSearcherArguments(db_path=db_path,
                                                seed=seed,
                                                data_pool_size=pool_size,
                                                optimal_subset_size=search_size,
                                                warmup_runs=warmup_runs,
                                                annealing_runs=annealing_runs,
                                                wandb_project=wandb_project,
                                                wandb_entity=wandb_entity
                                               )

In [6]:
processed_ds = preprocess.get_dataset(ds_name, training_args.model_card)
valid = processed_ds[valid_split]

Reusing dataset glue (/home/johnny/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/5 [00:00<?, ?it/s]

Loading cached processed dataset at /home/johnny/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-5e1b6e513b0be230.arrow
Loading cached processed dataset at /home/johnny/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-d50d011c86967268.arrow
Loading cached processed dataset at /home/johnny/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-f4beef46572a9827.arrow
Loading cached processed dataset at /home/johnny/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-eac0901845b0da03.arrow
Loading cached processed dataset at /home/johnny/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-992e78c958215b58.arrow


In [7]:
if test_dataset is None:
    test = processed_ds[test_split]
else:
    processed_test = preprocess.get_dataset(test_dataset, training_args.model_card)
    test = processed_test[test_split]

Reusing dataset hans (/home/johnny/.cache/huggingface/datasets/hans/plain_text/1.0.0/1bbcb735c482acd54f2e118074b59cfd2bf5f7a5a285d4d540d1e632216672ac)


  0%|          | 0/2 [00:00<?, ?it/s]

Loading cached processed dataset at /home/johnny/.cache/huggingface/datasets/hans/plain_text/1.0.0/1bbcb735c482acd54f2e118074b59cfd2bf5f7a5a285d4d540d1e632216672ac/cache-7eaedadf7f0b2cc0.arrow
Loading cached processed dataset at /home/johnny/.cache/huggingface/datasets/hans/plain_text/1.0.0/1bbcb735c482acd54f2e118074b59cfd2bf5f7a5a285d4d540d1e632216672ac/cache-5fd2fa11a6ce3173.arrow


In [8]:
subset_trainer = select.SubsetTrainer(
    params=training_args, valid_ds=valid, test_ds=test
)

In [9]:
data_pool = processed_ds["train"].shuffle(seed=searching_args.seed).select(range(searching_args.data_pool_size))

Loading cached shuffled indices for dataset at /home/johnny/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-0d3e1cebe9f0ca79.arrow


In [10]:
subset_searcher = select.WarmupAnnealingSearcher(subset_trainer=subset_trainer, params=searching_args, data_pool=data_pool)

In [11]:
subset_searcher.search_til_manual_termination()

wandb: Currently logged in as: johntzwei (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.13.3 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
2022-10-03 02:17:09.889446: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.1/lib64
2022-10-03 02:17:09.889493: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/3750 [00:00<?, ?it/s]

[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 

...

[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 

[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 

[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 

[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 1] [0, 0, 0, 

KeyboardInterrupt: 

In [ ]:
from subset_active_learning.active_learning.subset_classifier import get_df_from_db
df = get_df_from_db(db_path); df